#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [2]:
print("Before starting the entire loop. First Check what is the maximum size of the rock you can run the code for...")

Before starting the entire loop. First Check what is the maximum size of the rock you can run the code for...


#### Automate the GLSZM feature extraction process

In [12]:
%%time
print("Starting the automation loop for GLSZM feature extraction")

# All the greyscale images of various porous media types are named as data#.nrrd. 
# Iterate over the names to calculate the GLSZM features
for num_img in range(209, 210):
    
    # Create the variables for image and label
    global image_1, label_1

    # Path of the images used in REV analysis
    path ='D:\\Ankita PhD\\pyradiomics\\data\\data{}.nrrd'.format(num_img)
    
    # Read the nrrd image using the SITK library. Nrrd images are 8-bit images with intensity values between 0 to 255
    image_1 = sitk.ReadImage(path) 
    
    # Set the settings paramter needed for the bin_width and label
    # Bin_Width = 1 (as I don't want any further reduction in greyscale values and want to preserve the information)
    # label is a nrrd image that incorporates the window size and hence, gives a ROI for the GLSZM calculation.
    # The minimum size is set by the user and the mavimum size is the size of the image
    settings = {}
    settings['label'] = 255
    settings['binWidth'] = 1

    # Set-up logging for all the calculations
    log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\log{}.txt'.format(num_img)
    handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
    handler.setFormatter(formatter)
    radiomics.logger.addHandler(handler)

    # Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
    # Verbositiy level, the logger level will also determine the amount of information printed to the output
    radiomics.logger.setLevel(logging.DEBUG)

    # the size of the mask (same as label) should be the same as that of the image
    mask_x = image_1.GetWidth()
    mask_y = image_1.GetWidth()
    mask_z = image_1.GetWidth()

    #list for all the features being extracted
    v1 = []
    v2 = []
    v3 = []
    v4 = []
    v5 = []
    v6 = []
    v7 = []
    v8 = []
    v9 = []
    v10 = []
    v11 = []
    v12 = []
    v13 = []
    v14 = []
    v15 = []
    v16 = []
    v1_norm = []
    v3_norm = []
    v4_norm = []
    v5_norm = []
    v6_norm = []
    v7_norm = []
    v9_norm = []
    v12_norm = []
    v16_norm = []

    print("Starting the domain centre calculcation for image",str(num_img))
    
    # here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
    # sizes increase with every iteration
    u_window_size = [800] #2, 3, 4, 5, 6, 7, 8 , 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150, 180, 200, 250, 300, 310, 320, 330, 340, 350] #, 350, 400, 450, 500, 550, 600]#, 650, 700, 750, 800]
    
    for window_size in u_window_size:  

        i_loc = int((mask_x - window_size)/2)
        j_loc = int((mask_y - window_size)/2)
        k_loc = int((mask_z - window_size)/2)

        # Generate the mask
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc+window_size,j_loc:j_loc+window_size,k_loc:k_loc+window_size] = 255
        print("Window Location", str(i_loc), str(i_loc+window_size), str(j_loc), str(j_loc+window_size), str(k_loc), str(k_loc+window_size))
        print(str(full_mask.size))
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        label_1.CopyInformation(image_1)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        print("Cropped Mask and Image created for domain centre size =",str(window_size))
        
        import radiomics
        from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
        #calculate the original glszm
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        result = glszmFeatures.execute()
        
        print("Calculated GLSZM.....")
        glszm_org = glszmFeatures.P_glszm
        
        s_org = glszm_org.shape
        path_glszm_org ='D:\\Ankita PhD\\pyradiomics\\data\\data{}_glszm_org{}.csv'.format(num_img,window_size)
        pd.DataFrame(glszm_org[0, 0:s_org[1], 0:s_org[2]]).to_csv(path_glszm_org)

        #vector of all the gray-level valus present in the original ROI
        im_exp = sitk.GetArrayFromImage(croppedImage)
        iv_org =  np.unique(im_exp)#glszmFeatures.coefficients['ivector']
        iv_org = iv_org.astype(float)
        print(str(iv_org))
        iv_org = iv_org + 1
        print(str(iv_org))
        #Old jvector
        pd.DataFrame(iv_org).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data{}_iv_org{}.csv".format(num_img,window_size))

        #vector of all the size zones in the original GLSZM
        jv_org = glszmFeatures.coefficients['jvector']
        #Old jvector
        pd.DataFrame(jv_org).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data{}_jv_org{}.csv".format(num_img,window_size))
        
        # sum of all the size zones for all the graylevels of a particular size zones
        # for every column of GLSZM, add all the rows
        ps_org = glszmFeatures.coefficients['ps']
        
        #Remove the mineral or saturated phase if it exists FROM GLSZM
        if iv_org[iv_org.size - 1] == 256:
            print('Mineral phase is present because the graylevel of', iv_org[iv_org.size-1], 'exists.')
            mineralGrayLevels_idx = iv_org.size-1
            #Calculate the new GLSZM
            glszm_no_mineral = np.delete(glszm_org, mineralGrayLevels_idx, 1)
            s_nm = glszm_no_mineral.shape
            
            #Calculate the new ivector
            iv_new = iv_org[0:iv_org.size-1]  # shape (Ng,)
            print("iv_new", str(iv_new))
            #New jvector
            pd.DataFrame(iv_new).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data{}_iv_new{}.csv".format(num_img,window_size))                        
            
            #Calculate the new ps array with zero values
            glszm_2d_nm = glszm_no_mineral[0, 0:s_nm[1], 0:s_nm[2]]
            ps_new_zeros = np.sum(glszm_2d_nm, 0)
            ps_reshape = np.reshape(ps_new_zeros, (1,s_nm[2]))

            #Delete columns that specify zone sizes not present in the ROI
            emptyZoneSizes = np.where(np.sum(ps_reshape, 0) == 0)
            glszm_new = np.delete(glszm_no_mineral, emptyZoneSizes, 2)
            s_new = glszm_new.shape
            jv_new = np.delete(jv_org, emptyZoneSizes)
            #New jvector
            pd.DataFrame(jv_new).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data{}_jv_new{}.csv".format(num_img,window_size))
            ps_new = np.delete(ps_org, emptyZoneSizes, 1)
            
            # Get the number of zones in this GLSZM
            Nz_new = np.sum(glszm_new, (1, 2))  # shape (Nvox,)
            Nz_new[Nz_new == 0] = 1  # set sum to numpy.spacing(1) if sum is 0?

            # Get the number of voxels represented by this GLSZM: Multiply the zones by their size and sum them
            Np_new = np.sum(ps_new * jv_new[None, :], 1)  # shape (Nvox, )
            Np_new[Np_new == 0] = 1
            
            # Get the pg
            glszm_2d_new = glszm_new[0, 0:s_new[1], 0:s_new[2]]
            path_glszm_new ='D:\\Ankita PhD\\pyradiomics\\data\\data{}_glszm_new{}.csv'.format(num_img,window_size)
            pd.DataFrame(glszm_2d_new).to_csv(path_glszm_new)
            pg_reshape = np.sum(glszm_2d_new, 1)
            pg_new = np.reshape(pg_reshape, (1,s_new[1]))
            
        else:
            
            #just crab the coeffiecients as usual
            iv_new = iv_org
            jv_new = jv_org
            glszm_new = glszm_org
            Nz_new = glszmFeatures.coefficients['Nz']
            Np_new = glszmFeatures.coefficients['Np']
            pg_new = glszmFeatures.coefficients['pg']
            ps_new = glszmFeatures.coefficients['ps']
                    
        jvector = jv_new
        ivector = iv_new
        ps = ps_new
        pg = pg_new
        Np = Np_new
        Nz = Nz_new
        pg_norm = pg/Nz[:, None]
        ps_norm = ps/Nz[:, None]
        eps = np.spacing(1)
        glszm = glszm_new
        print('The shape of the old glszm is', str(glszm_org.shape), '. The shape of the new glszm is', str(glszm.shape))
        glszm_norm = glszm_new/Nz[:, None, None]
        
        print("GLSZM processing complete ...")
        
        #del glszm_org, glszm_new, iv_org, jv_org, ps_org, mineralGrayLevels_idx, glszm_no_mineral, s_nm, iv_new, glszm_2d_nm
        #del ps_new_zeros, ps_reshape, emptyZoneSizes, glszm_new, s_new, jv_new, ps_new, Nz_new, Np_new, glszm_2d_nm, pg_reshape
        #del pg_new
            
        #Calculate glszm features and normalize the features
        print("Initiating GLSZM feature calculation...... ")
        
        sae = np.sum(ps / (jvector[None, :] ** 2), 1) / Nz #small area emphaisis
        lae = np.sum(ps * (jvector[None, :] ** 2), 1) / Nz #large area emphaisis
        iv = np.sum(pg ** 2, 1) / Nz #graylevel non-uniformity
        ivn = np.sum(pg ** 2, 1) / Nz ** 2 #graylevel non-uniformity normalized
        szv = np.sum(ps ** 2, 1) / Nz #size zone uniformity
        szvn = np.sum(ps ** 2, 1) / Nz ** 2 #size zone uniformity normalised
        zp = Nz / Np # zone percentage
        u_i = np.sum(pg_norm * ivector[None, :], 1, keepdims=True) # mean calculation for graylevel variance
        glv = np.sum(pg_norm * (ivector[None, :] - u_i) ** 2, 1) # graylevel variance
        u_j = np.sum(ps_norm * jvector[None, :], 1, keepdims=True) #mean caluclation of zone variance
        zv = np.sum(ps_norm * (jvector[None, :] - u_j) ** 2, 1) #zone variance
        ze = -np.sum(glszm_norm * np.log2(glszm_norm + eps), (1, 2)) #zone entropy
        lie = np.sum(pg / (ivector[None, :] ** 2), 1) / Nz #Low graylevel zone emphaisis
        hie = np.sum(pg * (ivector[None, :] ** 2), 1) / Nz #high graylevel zone emphaisis
        lisae = np.sum(glszm / ((ivector[None, :, None] ** 2) * (jvector[None, None, :] ** 2)), (1, 2)) / Nz #small area low graylevel emphaisis
        hisae = np.sum(glszm * (ivector[None, :, None] ** 2) / (jvector[None, None, :] ** 2), (1, 2)) / Nz #small area high graylevel emphaisis
        lilae = np.sum(glszm * (jvector[None, None, :] ** 2) / (ivector[None, :, None] ** 2), (1, 2)) / Nz #large area low gray level emphaisis
        hilae = np.sum(glszm * (ivector[None, :, None] ** 2) * (jvector[None, None, :] ** 2), (1, 2)) / Nz #large area high gray level emphaisis
        
        #normalise features by the number of voxels
        iv_np = iv/Np
        glv_np = glv/Np
        hie_np = hie/Np
        lae_np = lae/Np
        hilae_np = hilae/Np
        lilae_np = lilae/Np
        szv_np = szv/Np
        hisae_np = hisae/Np
        zv_np = zv/Np
        
        #append values
        v1.append(iv[0])
        v1_norm.append(iv_np[0])
        v2.append(ivn[0])
        v3.append(glv[0])
        v3_norm.append(glv_np[0])
        v4.append(hie[0])
        v4_norm.append(hie_np[0])
        v5.append(lae[0])
        v5_norm.append(lae_np[0])
        v6.append(hilae[0])
        v6_norm.append(hilae_np[0])
        v7.append(lilae[0])
        v7_norm.append(lilae_np[0])
        v8.append(lie[0])
        v9.append(szv[0])
        v9_norm.append(szv_np[0])
        v10.append(szvn[0])
        v11.append(sae[0])
        v12.append(hisae[0])
        v12_norm.append(hisae_np[0])
        v13.append(lisae[0])
        v14.append(ze[0])
        v15.append(zp[0])
        v16.append(zv[0])
        v16_norm.append(zv_np[0])
        
        del sae, lae, iv, ivn, szv, szvn, zp, u_i, glv, u_j, zv, ze, lie, hie, lisae, hisae, lilae, hilae
        del iv_np, glv_np, hie_np, lae_np, hilae_np, lilae_np, szv_np, hisae_np, zv_np
        
        
        print("Completed calculations for domain centre size =", str(window_size))
    
    # Create a data-frame to store all the GLSZM features for the image    
    my_df = pd.DataFrame(list(zip(u_window_size, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, \
                                  v1_norm, v3_norm, v4_norm, v5_norm, v6_norm, v7_norm, v9_norm, v12_norm, v16_norm)))
    my_df = my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance", 17: "Norm GrayLevelNonUniformity", \
                                   18: "Norm GrayLevelVariance", 19: "Norm HighGrayLevelZoneEmphasis", 20: "Norm LargeAreaEmphasis", \
                                   21: "Norm LargeAreaHighGrayLevelEmphasis", 22: "Norm LargeAreaLowGrayLevelEmphasis", \
                                   23: "Norm SizeZoneNonUniformity", 24:"Norm SmallAreaHighGrayLevelEmphasis", 25: "Norm ZoneVariance"})    
    
    name_csv = 'glszm_crrtd_dc_data{}.csv'.format(num_img)
    my_df.to_csv(name_csv, index=False, header=True)
    
    print("Stored all the GLSZM features for image", str(num_img), "in a csv file")
    
    #free the memory and delete all the variables
    del image_1
    del my_df
    del glszmFeatures

Starting the automation loop for GLSZM feature extraction
Starting the domain centre calculcation for image 209
Window Location 0 800 0 800 0 800
512000000
Cropped Mask and Image created for domain centre size = 800
Calculated GLSZM.....
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 1

In [13]:
high_lae = glszm * (ivector[None, :, None] ** 2) * (jvector[None, None, :] ** 2)
high_lae_2D = high_lae[0, 0:s_new[1], 0:s_new[2]]
np.savetxt("HIGHNAV209_800.csv", high_lae_2D, delimiter=",")         
                                                    
low_lae = glszm * (jvector[None, None, :] ** 2) / (ivector[None, :, None] ** 2)
low_lae_2D = low_lae[0, 0:s_new[1], 0:s_new[2]]
np.savetxt("LOWNAV209_800.csv", low_lae_2D, delimiter=",")                                                                                                                 